In [28]:
import pandas as pd
from sklearn.ensemble import *

In [48]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [49]:
train[:5]

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,SignFacing (Target)
0,2c9180975a056a64015a1e0a52e57021,Rear,195,1.02,46,45,Rear
1,2c9180975a056a64015a1e17b32171e4,Rear,203,1.09,59,54,Rear
2,2c9180975a056a64015a1de4deb16bd5,Front,26,0.96,104,108,Front
3,2c9180975a056a64015a1de4deb16bdd,Rear,199,0.81,38,47,Rear
4,2c9180975a056a64015a1de4deb16bd6,Rear,208,0.93,54,58,Rear


In [50]:
train.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,SignFacing (Target)
0,2c9180975a056a64015a1e0a52e57021,Rear,195,1.02,46,45,Rear
1,2c9180975a056a64015a1e17b32171e4,Rear,203,1.09,59,54,Rear
2,2c9180975a056a64015a1de4deb16bd5,Front,26,0.96,104,108,Front
3,2c9180975a056a64015a1de4deb16bdd,Rear,199,0.81,38,47,Rear
4,2c9180975a056a64015a1de4deb16bd6,Rear,208,0.93,54,58,Rear


In [51]:
test.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight
0,2c9180975a056a64015a1e10d3f270fe,Right,67,0.63,107,169
1,2c9180975a056a64015a1de4deb16bdc,Front,16,0.88,61,69
2,2c9180975a056a64015a1e0e70ea70ce,Right,44,1.15,232,202
3,2c9180975a056a64015a1dfed0c46ec6,Right,50,1.10,137,125
4,2c9180975a056a64015a1dfed0c46ec7,Front,30,0.95,99,104


In [52]:
train['DetectedCamera'].value_counts()

Front    10910
Right    10516
Left      9298
Rear      7761
Name: DetectedCamera, dtype: int64

In [53]:
#encode as integer
mapping = {'Front':0, 'Right':1, 'Left':2, 'Rear':3}
train = train.replace({'DetectedCamera':mapping})
test = test.replace({'DetectedCamera':mapping})

In [54]:
train.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,SignFacing (Target)
0,2c9180975a056a64015a1e0a52e57021,3,195,1.02,46,45,Rear
1,2c9180975a056a64015a1e17b32171e4,3,203,1.09,59,54,Rear
2,2c9180975a056a64015a1de4deb16bd5,0,26,0.96,104,108,Front
3,2c9180975a056a64015a1de4deb16bdd,3,199,0.81,38,47,Rear
4,2c9180975a056a64015a1de4deb16bd6,3,208,0.93,54,58,Rear


In [55]:
#renaming column
train.rename(columns = {'SignFacing (Target)': 'Target'}, inplace=True)

In [56]:
#encode Target Variable based on sample submission file
mapping = {'Front':0, 'Left':1, 'Rear':2, 'Right':3}
train = train.replace({'Target':mapping})

In [57]:
#target variable
y_train = train['Target']
test_id = test['Id']

In [58]:
#drop columns
train.drop(['Target','Id'], inplace=True, axis=1)
test.drop('Id',inplace=True,axis=1)

In [60]:
train[:5]

,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight
0,3,195,1.02,46,45
1,3,203,1.09,59,54
2,0,26,0.96,104,108
3,3,199,0.81,38,47
4,3,208,0.93,54,58


In [61]:
test.head()

,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight
0,1,67,0.63,107,169
1,0,16,0.88,61,69
2,1,44,1.15,232,202
3,1,50,1.10,137,125
4,0,30,0.95,99,104


In [23]:
#train model
clf = RandomForestClassifier(n_estimators=500,max_features=3,min_samples_split=5,oob_score=True)
clf.fit(train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [24]:
#predict on test data
pred = clf.predict_proba(test)

In [25]:
#write submission file and submit
columns = ['Front','Left','Rear','Right']
sub = pd.DataFrame(data=pred, columns=columns)
sub['Id'] = test_id
sub = sub[['Id','Front','Left','Rear','Right']]
sub.to_csv("sub_rfih.csv", index=False) #99.82483

In [34]:
#train second model
sclf = GradientBoostingClassifier(n_estimators=600,max_features=3,min_samples_split=5)
sclf.fit(train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=3, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=5,
              min_weight_fraction_leaf=0.0, n_estimators=600,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [35]:
#predict on test data
pred2 = sclf.predict_proba(test)


#write submission file and submit
columns = ['Front','Left','Rear','Right']
sub2 = pd.DataFrame(data=pred2, columns=columns)
sub2['Id'] = test_id
sub2 = sub[['Id','Front','Left','Rear','Right']]
sub2.to_csv("sub_2.csv", index=False) #

In [36]:
pred2

array([[9.96496069e-01, 1.95939578e-04, 1.82050363e-03, 1.48748735e-03],
       [9.98342249e-01, 5.95486389e-04, 3.40403584e-04, 7.21860911e-04],
       [9.99001438e-01, 9.64659831e-05, 5.97033632e-04, 3.05062559e-04],
       ...,
       [3.45369536e-04, 3.61308157e-04, 9.98827202e-01, 4.66120700e-04],
       [7.83296330e-03, 2.77013437e-03, 9.89153216e-01, 2.43685855e-04],
       [2.77180912e-04, 1.03776768e-03, 9.98361355e-01, 3.23696741e-04]])